In [1]:
import pandas as pd
import random
import os
import ast

# Master Movie Data Manipulation Notebook

How this notebook could work is that we iterate/clean on the 'meta' csv in pandas, and then save it in the repo after we have added the new/corrected/cleaned data

In [2]:
#### change pathname to your clean dataset
meta = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/movies_metadata.csv')
#meta.head(2).T

/Users/nickparker/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(meta.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [4]:
### Adding profit and use of Budget Columns
meta['profit'] = meta['revenue'] - pd.to_numeric(meta['budget'],errors='coerce')
meta['useOfBudget'] = meta['revenue'] / (pd.to_numeric(meta['budget'],errors='coerce') + 000.1)

In [5]:
##Adding production company
#Added Production Companies as a trend
productionCompanies1 = []
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies1.append(ast.literal_eval(meta['production_companies'][rowNum])[0]['name'])
    except:
        productionCompanies1.append('nan')
        
productionCompanies2 = []        
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies2.append(ast.literal_eval(meta['production_companies'][rowNum])[1]['name'])
    except:
        productionCompanies2.append('nan')

productionCompanies3 = []        
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies3.append(ast.literal_eval(meta['production_companies'][rowNum])[2]['name'])
    except:
        productionCompanies3.append('nan')        

productionCompanies4 = []        
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies4.append(ast.literal_eval(meta['production_companies'][rowNum])[3]['name'])
    except:
        productionCompanies4.append('nan')

productionCompanies5 = []        
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies5.append(ast.literal_eval(meta['production_companies'][rowNum])[4]['name'])
    except:
        productionCompanies5.append('nan')

productionCompanies6 = []        
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies6.append(ast.literal_eval(meta['production_companies'][rowNum])[5]['name'])
    except:
        productionCompanies6.append('nan')

meta['ProductionCo_1'] = productionCompanies1
meta['ProductionCo_2'] = productionCompanies2
meta['ProductionCo_3'] = productionCompanies3
meta['ProductionCo_4'] = productionCompanies4
meta['ProductionCo_5'] = productionCompanies5
meta['ProductionCo_6'] = productionCompanies6

In [6]:
##Adding genre
genre1 = []
for rowNum in range(meta.shape[0]):
    try: 
        genre1.append(ast.literal_eval(meta['genres'][rowNum])[0]['name'])
    except:
        genre1.append('nan')
        
genre2 = []        
for rowNum in range(meta.shape[0]):
    try: 
        genre2.append(ast.literal_eval(meta['genres'][rowNum])[1]['name'])
    except:
        genre2.append('nan')

genre3 = []        
for rowNum in range(meta.shape[0]):
    try: 
        genre3.append(ast.literal_eval(meta['genres'][rowNum])[2]['name'])
    except:
        genre3.append('nan')
meta['genre_1'] = genre1
meta['genre_2'] = genre2
meta['genre_3'] = genre3

## Merging

In [7]:
## Don't run this cell unless you want to merge in ratings (big file)
#Also, update filepath
ratings = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/ratings.csv')
links = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/links.csv')

ratings = ratings.groupby('movieId').agg({'rating':['mean','count']}).reset_index() #gets average rating
ratings.columns = ['movieId','rating','rateCounts']
linkAvgRating = pd.merge(links,ratings,on='movieId',how='left')

#credits = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/credits.csv')
#credits.columns = ['cast','crew','movieId']
#linkRatCred = pd.merge(linkAvgRating,credits,on='movieId',how='left')

meta['imdbId'] = pd.to_numeric(meta['imdb_id'].str[2:])
meta = pd.merge(meta,linkAvgRating,on='imdbId',how='left')

## Making Flops dataframe

In [8]:
flop = meta.loc[((meta['revenue'] > 0) & pd.to_numeric(meta['budget'],errors='coerce' != 0))]
flop['flop'] = 0
flop.loc[(flop['profit'] < 0),'flop'] = 1
flop.shape

/Users/nickparker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/nickparker/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(5381, 40)

## Making the df longer based on genre

In [9]:
flop.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'profit', 'useOfBudget', 'ProductionCo_1',
       'ProductionCo_2', 'ProductionCo_3', 'ProductionCo_4', 'ProductionCo_5',
       'ProductionCo_6', 'genre_1', 'genre_2', 'genre_3', 'imdbId', 'movieId',
       'tmdbId', 'rating', 'flop'],
      dtype='object')

In [9]:
#metaTest = meta[['id','genre_1','genre_2','genre_3']]
#pd.wide_to_long(metaTest, stubnames='genre', i=['imdb_id'], j='genreNo')
#pd.melt(metaTest,id_vars=['imdb_id'],var_name='metrics', value_name='values')
long_df = pd.wide_to_long(flop.reset_index(), 
                          i=['index'], 
                          j='genreNo',
                          stubnames='genre',
                          sep="_")
long_df.reset_index(inplace=True)
long_df.shape

(16143, 40)

## Making the df longer based on prod company

In [10]:
long_df = pd.wide_to_long(flop.reset_index(), 
                          i=['index'], 
                          j='companyNo',
                          stubnames='ProductionCo',
                          sep="_")
long_df.shape

(32286, 35)

In [11]:
long_df.reset_index(inplace=True)

display = long_df[['title','id','companyNo','ProductionCo']]

display.sort_values('id').head(100)

,title,id,companyNo,ProductionCo
17079,"Lock, Stock and Two Smoking Barrels",100,4,SKA Films
22460,"Lock, Stock and Two Smoking Barrels",100,5,The Steve Tisch Company
11698,"Lock, Stock and Two Smoking Barrels",100,3,PolyGram Filmed Entertainment
27841,"Lock, Stock and Two Smoking Barrels",100,6,nan
6317,"Lock, Stock and Two Smoking Barrels",100,2,Summit Entertainment
936,"Lock, Stock and Two Smoking Barrels",100,1,Handmade Films Ltd.
27416,The Saint,10003,6,nan
16654,The Saint,10003,4,nan
511,The Saint,10003,1,Paramount Pictures
11273,The Saint,10003,3,Rysher Entertainment


/Users/nickparker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
flop.shape

(5381, 36)

# Export

#### When you are done with the data manipulation, save as csv in our manipulated data folder

In [10]:
meta.head(2).T

,0,1
adult,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN
budget,30000000,65000000
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
homepage,http://toystory.disney.com/toy-story,NaN
id,862,8844
imdb_id,tt0114709,tt0113497
original_language,en,en
original_title,Toy Story,Jumanji
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...


In [9]:
export_csv = meta.to_csv (r'movie_dataset_edited/meta.csv', header=True)
#Don't forget to add '.csv' at the end of the path

In [37]:
export_csv = flop.to_csv (r'movie_dataset_edited/flopDataSet.csv', header=True)


In [11]:
export_csv = long_df.to_csv (r'movie_dataset_edited/flopTripleByGenre.csv', header=True)


In [10]:
long_df.shape

(16143, 40)